In [1]:
import pandas as pd

CREATING INDIVIDUAL DATAFRAMES

In [4]:
amazon_df = pd.read_csv('amazon_reviews.csv')
twitter_df = pd.read_csv('Tweets.csv')
yelp_df = pd.read_csv('yelp.csv')
amazon_df.isnull().sum()

Unnamed: 0              0
reviewerName            1
overall                 0
reviewText              1
reviewTime              0
day_diff                0
helpful_yes             0
helpful_no              0
total_vote              0
score_pos_neg_diff      0
score_average_rating    0
wilson_lower_bound      0
dtype: int64

PERFORMING DATA LOADING BY CHOOSING ONLY REQUIRED COLUMNS ON DATAFRAME

In [3]:
def data_loading(df,features,label):
    X = df[features]
    y = df[label]
    X.columns = ['review_statement']
    y.columns = ['result_analysis']
    return X,y

In [5]:
features = ['reviewText']
label = ['overall']
X1,y1 = data_loading(amazon_df,features,label)
X1

,review_statement
0,No issues.
1,"Purchased this for my device, it worked as adv..."
2,it works as expected. I should have sprung for...
3,This think has worked out great.Had a diff. br...
4,"Bought it with Retail Packaging, arrived legit..."
...,...
4910,I bought this Sandisk 16GB Class 10 to use wit...
4911,Used this for extending the capabilities of my...
4912,Great card that is very fast and reliable. It ...
4913,Good amount of space for the stuff I want to d...


In [6]:
features = ['text']
label = ['sentiment']
X2,y2 = data_loading(twitter_df,features,label)
X2

,review_statement
0,"I`d have responded, if I were going"
1,Sooo SAD I will miss you here in San Diego!!!
2,my boss is bullying me...
3,what interview! leave me alone
4,"Sons of ****, why couldn`t they put them on t..."
...,...
27476,wish we could come see u on Denver husband l...
27477,I`ve wondered about rake to. The client has ...
27478,Yay good for both of you. Enjoy the break - y...
27479,But it was worth it ****.


In [7]:
features = ['text']
label = ['stars']
X3,y3 = data_loading(yelp_df,features,label)
X3

,review_statement
0,My wife took me here on my birthday for breakf...
1,I have no idea why some people give bad review...
2,love the gyro plate. Rice is so good and I als...
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!..."
4,General Manager Scott Petello is a good egg!!!...
...,...
9995,First visit...Had lunch here today - used my G...
9996,Should be called house of deliciousness!\n\nI ...
9997,I recently visited Olive and Ivy for business ...
9998,My nephew just moved to Scottsdale recently so...


DEALING WITH LABELS

In [8]:
y1['result_analysis'].unique()

array([4., 5., 3., 1., 2.])

In [9]:
for x in y1['result_analysis'].unique():
    if int(x)<=2:
        y1.loc[y1['result_analysis']==x,'result_analysis'] = 'negative'
    elif int(x)==3:
        y1.loc[y1['result_analysis']==x,'result_analysis'] = 'neutral'
    elif int(x)>=4:
        y1.loc[y1['result_analysis']==x,'result_analysis'] = 'positive'


/tmp/ipykernel_20267/1006990375.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y1.loc[y1['result_analysis']==x,'result_analysis'] = 'positive'
/tmp/ipykernel_20267/1006990375.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y1.loc[y1['result_analysis']==x,'result_analysis'] = 'positive'
/tmp/ipykernel_20267/1006990375.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y1

In [10]:
y2['result_analysis'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [11]:
y3['result_analysis'].unique()


array([5, 4, 2, 3, 1])

In [12]:
for x in y3['result_analysis'].unique():
    if int(x)<=2:
        y3.loc[y3['result_analysis']==x,'result_analysis'] = 'negative'
    elif int(x)==3:
        y3.loc[y3['result_analysis']==x,'result_analysis'] = 'neutral'
    elif int(x)>=4:
        y3.loc[y3['result_analysis']==x,'result_analysis'] = 'positive'

/tmp/ipykernel_20267/4203180970.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y3.loc[y3['result_analysis']==x,'result_analysis'] = 'positive'
/tmp/ipykernel_20267/4203180970.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y3.loc[y3['result_analysis']==x,'result_analysis'] = 'positive'
/tmp/ipykernel_20267/4203180970.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y3

In [13]:

X1.isnull().sum()

review_statement    1
dtype: int64